<h1> Run to Mount Drive Instance with Current Data (Must Run First) <h1>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

<h1> Format (.txt) Files to (.csv) Data <h1>


The code below takes the folder of stock indices and converts the entire folder of txt files to a single csv.

In [17]:
import os
import pandas as pd

# Define the folder path containing the .txt files and the output CSV file.
FOLDER_PATH = r"C:\Users\kjbsh\Documents\CSC442-Project-Econ-vs-Stock-Market\originalDataSets\world\stooq stocks indices"  # Change this to your folder location
OUTPUT_FILE = r"C:\Users\kjbsh\Documents\StockOut.csv"  # Change the output file name if needed

def merge_txt_to_csv(folder_path, output_file):
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    merged_data = []

    if not all_files:
        print("No text files found in the folder.")
        return

    for file in all_files:
        file_path = os.path.join(folder_path, file)
        try:
            # Read the file into a DataFrame
            df = pd.read_csv(file_path, delimiter=',', header=0)  # Adjust delimiter if needed
            df['Source'] = file  # Add a column to track source file name
            merged_data.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    if merged_data:
        stock_df = pd.concat(merged_data, ignore_index=True)
        stock_df.to_csv(output_file, index=False)
        print(f"Merged CSV saved as: {output_file}")
        return stock_df
    else:
        print("No valid data to merge.")

# Run the function
stock_df = merge_txt_to_csv(FOLDER_PATH, OUTPUT_FILE)

Merged CSV saved as: C:\Users\kjbsh\Documents\StockOut.csv


<h1> Selects Valid Countries from GEI Data <h1>

<h2> Countries: Germany, China(HK), Hungary, Japan, Poland, UK, US <h2

In [18]:
import pandas as pd


# 1. Read in the econ data

df = pd.read_csv(r"c:\Users\kjbsh\Documents\CSC442-Project-Econ-vs-Stock-Market\originalDataSets\Global Economy Indicators.csv")

# Strip whitespace from column names
df.columns = df.columns.str.strip()
# Strip whitespace from Country names
df["Country"] = df["Country"].str.strip()
# Print column names to verify
print(df.columns)

# 2. Define the list of valid countries
valid_countries = [
    "Germany",                  # ^de
    "China, Hong Kong SAR",     # ^hk
    "Hungary",                  # ^hu
    "Japan",                    # ^jp
    "Poland",                   # ^pl, ^pl20, ^plnc, ^plws
    "United Kingdom",           # ^uk
    "United States"             # ^us, ^usnm, ^usnq, ^usns
]

# 3. Filter the DataFrame so only rows with these countries remain
gei_df = df[df["Country"].isin(valid_countries)]

# 4. Write filtered data to a new CSV
gei_df.to_csv(r"c:\Users\kjbsh\Documents\econData.csv", index=False)

print("Filtered CSV saved as econData.csv")

Index(['CountryID', 'Country', 'Year', 'AMA exchange rate',
       'IMF based exchange rate', 'Population', 'Currency', 'Per capita GNI',
       'Agriculture, hunting, forestry, fishing (ISIC A-B)',
       'Changes in inventories', 'Construction (ISIC F)',
       'Exports of goods and services', 'Final consumption expenditure',
       'General government final consumption expenditure',
       'Gross capital formation',
       'Gross fixed capital formation (including Acquisitions less disposals of valuables)',
       'Household consumption expenditure (including Non-profit institutions serving households)',
       'Imports of goods and services', 'Manufacturing (ISIC D)',
       'Mining, Manufacturing, Utilities (ISIC C-E)',
       'Other Activities (ISIC J-P)', 'Total Value Added',
       'Transport, storage and communication (ISIC I)',
       'Wholesale, retail trade, restaurants and hotels (ISIC G-H)',
       'Gross National Income(GNI) in USD', 'Gross Domestic Product (GDP)'],
    

<h1> Data Merging <h1>

In [21]:
import pandas as pd

# READ BOTH DATASETS
# Replace these paths with the actual file locations
gei_df = pd.read_csv(r"c:\Users\kjbsh\Documents\econData.csv")
stock_df = pd.read_csv(r"c:\Users\kjbsh\Documents\StockOut.csv")

# PRUNE GEI COLUMNS (DROP THE ONES YOU DON'T WANT)
columns_to_drop = [
    "AMA exchange rate",
    "Changes in inventories",
    "Construction (ISIC F)",
    "Gross capital formation",
    "Gross fixed capital formation (including Acquisitions less disposals of valuables)",
    "Household consumption expenditure (including Non-profit institutions serving households)",
    "Manufacturing (ISIC D)",
    "Other Activities (ISIC J-P)",
    "Total Value Added",
]
gei_df.drop(columns=[c for c in columns_to_drop if c in gei_df.columns],
            inplace=True, errors="ignore")

# KEEP ONLY <TICKER>, <DATE>, <VOL>, <CLOSE> IN THE STOCK DATA
stock_df = stock_df[["<TICKER>", "<DATE>", "<CLOSE>", "<VOL>"]]

# CONVERT <DATE> TO DATETIME, EXTRACT YEAR/QUARTER, PICK LAST ROW/Q
# Parse <DATE> = YYYYMMDD
stock_df["Date"] = pd.to_datetime(stock_df["<DATE>"], format="%Y%m%d", errors="coerce")

# Extract the Year
stock_df["Year"] = stock_df["Date"].dt.year

# Filter to ignore 2022–2024, keep only Year <= 2021
stock_df = stock_df.query("Year <= 2021")

# Determine the quarter
stock_df["Quarter"] = stock_df["Date"].dt.to_period("Q")

# Sort to ensure tail(1) picks the last date in each quarter
stock_df.sort_values(by=["<TICKER>", "Date"], inplace=True)

# For each TICKER / Year / Quarter, pick the last entry
stock_quarterly = (
    stock_df
    .groupby(["<TICKER>", "Year", "Quarter"], as_index=False)
    .tail(1)
)

# MAP TICKERS TO COUNTRIES
mapping_df = pd.DataFrame({
    "Country": [
        "Germany", 
        "China, Hong Kong SAR", 
        "Hungary",
        "Japan",
        "Poland", "Poland", "Poland", "Poland",
        "United Kingdom",
        "United States", "United States", "United States", "United States"
    ],
    "<TICKER>": [
        "^_DE", "^_HK", "^_HU", "^_JP",
        "^_PL", "^_PL20", "^_PLNC", "^_PLWS",
        "^_UK",
        "^_US", "^_USNM", "^_USNQ", "^_USNS"
    ]
})

stock_quarterly = stock_quarterly.merge(mapping_df, on="<TICKER>", how="left")

# REORDER COLUMNS
desired_order = [
    "Country", 
    "<TICKER>", "<DATE>", "<CLOSE>", "<VOL>",
    "Date", "Year", "Quarter", 
    "CountryID",
    "IMF based exchange rate",
    "Population",
    "Currency",
    "Per capita GNI",
    "Exports of goods and services",
    "Final consumption expenditure",
    "General government final consumption expenditure",
    "Imports of goods and services",
    "Gross National Income(GNI) in USD",
    "Gross Domestic Product (GDP)"
]

# Filter only those columns that actually exist in merged_df
final_columns = [col for col in desired_order if col in merged_df.columns]

merged_df = merged_df[final_columns]

# OUTPUT FINAL MERGED DATA AS CSV
merged_df.to_csv(r"c:\Users\kjbsh\Documents\CSC442-Project-Econ-vs-Stock-Market\finalMerged", index=False)

# preview the first few rows
print(merged_df.head())


   Country <TICKER>    <DATE>  <CLOSE>        <VOL>       Date  Year Quarter  \
0  Germany     ^_DE  20160331   964.04  123999909.0 2016-03-31  2016  2016Q1   
1  Germany     ^_DE  20160630   928.32  184567273.0 2016-06-30  2016  2016Q2   
2  Germany     ^_DE  20160930   957.00  232584243.0 2016-09-30  2016  2016Q3   
3  Germany     ^_DE  20161230   945.10   61221620.0 2016-12-30  2016  2016Q4   
4  Germany     ^_DE  20170331  1046.19  162410852.0 2017-03-31  2017  2017Q1   

   CountryID  IMF based exchange rate  Population Currency  Per capita GNI  \
0        276                 0.904035    82331423    Euro            43161   
1        276                 0.904035    82331423    Euro            43161   
2        276                 0.904035    82331423    Euro            43161   
3        276                 0.904035    82331423    Euro            43161   
4        276                 0.885206    82624374    Euro            45735   

   Exports of goods and services  Final consumptio